In [1]:
import os
import glob
import sys
import pandas as pd
import numpy as np
# plotting modules
import matplotlib as mpl
mpl.use('nbagg')
import seaborn as sns
import pylab as pl
# import some custom funcs|
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utils as util
import behavior as butil

In [2]:
%matplotlib notebook

In [3]:
util.set_sns_style(style='dark') # plotting settings i like for Nbs

In [4]:
# Set rootdir - I usually set this to the parent dir of all the data
rootdir = '/Users/julianarhee/Library/CloudStorage/GoogleDrive-edge.tracking.ru@gmail.com\
/My Drive/Edge_Tracking/Data'

# Specify experiment to analyze
# experiment = 'stripgrid_hdc-sytGCaMP7f_2p/20221102' 
experiment = 'HDC_inhibition_CM' 
src_dir = os.path.join(rootdir, experiment)

# Create a save dir for figures wherever you want
# I create a separate dir since I don't want to mess with Andy's existing organization
save_dir = '/Users/julianarhee/Documents/rutalab/data/figures/{}'.format(experiment)
save_dir = save_dir.replace(" ", "")
print("Saving figures to:{}    {}".format('\n', save_dir))
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Create a label/ID for figures so you always know where the plotted data came from
fig_id = src_dir.split('/My Drive')[1] #, experiment) 

Saving figures to:
    /Users/julianarhee/Documents/rutalab/data/figures/HDC_inhibition_CM


In [ ]:
os,

In [19]:
# get a list of all the data files
fly='Fly1'
log_files = sorted([k for k in glob.glob(os.path.join(src_dir, '*{}*.log'.format(fly)))\
                if 'lossed tracking' not in k], key=util.natsort)
print("Found {} tracking files.".format(len(log_files)))
for i, fn in enumerate(log_files):
    print("{}: {}".format(i, os.path.split(fn)[-1]))

Found 4 tracking files.
0: 11012022-110656_HdC_inhibition_thresh500_Fly1.log
1: 11012022-113023_HdC_inhibition_nothresh_Fly1.log
2: 11092022-151404_HDC_inhibition_nothresh_Fly1.log
3: 11092022-151958_HDC_inhibition_outside_500thresh_Fly1.log


In [20]:
file_ix = 1 # select a file
fpath = log_files[file_ix]
print("Selected: {}".format(fpath))

Selected: /Users/julianarhee/Library/CloudStorage/GoogleDrive-edge.tracking.ru@gmail.com/My Drive/Edge_Tracking/Data/HDC_inhibition_CM/11012022-113023_HdC_inhibition_nothresh_Fly1.log


In [21]:
importlib.reload(butil)

<module 'behavior' from '/Users/julianarhee/Repositories/plume-tracking/behavior.py'>

In [22]:
# try to parse experiment details from the filename
exp, datestr, fid, cond = butil.parse_info_from_file(fpath)
exp, datestr, fid, cond

('HDC_inhibition_CM', '11012022-113023', 'fly1', 'hdc_inhibition_nothresh')

In [23]:
# try to parse experiment details from the filename
exp, datestr, fid, cond = butil.parse_info_from_file(fpath)
# load and process the csv data  
df0 = butil.load_dataframe(fpath, mfc_id=None, verbose=False, cond=None)
print('Experiment: {}{}Fly ID: {}{}Condition: {}'.format(exp, '\n', fid, '\n', cond))
fly_id = df0['fly_id'].unique()[0]
df0.head()

TimeoutError: [Errno 60] Operation timed out

In [ ]:
# get experimentally determined odor boundaries:
odor_width=50
grid_sep=200 # only matters for 2p
ogrid = butil.get_odor_grid(df0, 
                            odor_width=odor_width, grid_sep=grid_sep,
                            use_crossings=True, verbose=False)
(odor_xmin, odor_xmax), = ogrid.values()
print(ogrid)

In [ ]:
# Set some plotting params 
hue_varname='led_on'
palette={True: 'r', False: 'w'}
start_at_odor = True
odor_width=10
odor_lc='lightgray'
odor_lw=0.5
# ---------------------------------------------------------------------
fig, ax = pl.subplots()
sns.scatterplot(data=df0, x="ft_posx", y="ft_posy", ax=ax, 
                hue=hue_varname, s=0.5, edgecolor='none', palette=palette)
butil.plot_odor_corridor(ax, odor_xmin=odor_xmin, odor_xmax=odor_xmax)
ax.legend(bbox_to_anchor=(1,1), loc='upper left', title=hue_varname)
ax.set_title(fly_id)
pl.subplots_adjust(left=0.2, right=0.8)
# Center corridor
xmax = np.ceil(df0['ft_posx'].abs().max())
ax.set_xlim([-xmax-10, xmax+10])
# label figure and save
util.label_figure(fig, fig_id)
figname = 'trajectory_{}'.format(fly_id)
pl.savefig(os.path.join(save_dir, '{}.png'.format(figname))) #, dpi=dpi)
print(os.path.join(save_dir, '{}.png'.format(figname)))